# RRAP-IS Authentication Demo Notebook

> A tutorial of RRAP authentication using Jupyter notebooks.

- toc: true 
- badges: true
- comments: true
- categories: [jupyter]

## About

This notebook is a demonstration of authenticating with the RRAP-IS system.  

#hide

Handy links
https://github.com/gbrrestoration/rrap-is-proto/blob/main/utilities/packages/python/keycloak-utils/KeycloakRestUtilities/TokenManager.py

https://github.com/gbrrestoration/rrap-is-proto/blob/main/admin-tooling/data-store/import_db.py
Construct Json directly

APIs:
https://data-api.testing.rrap-is.com/docs
https://registry-api.testing.rrap-is.com/docs
https://prov-api.testing.rrap-is.com/docs

Auth:
'Authorization' : 'Bearer <JWT token>'
https://data-api.testing.rrap-is.com/check-access/check-general-access
https://data-api.testing.rrap-is.com/

token:
https://data.testing.rrap-is.com/

Maintain session
https://github.com/gbrrestoration/rrap-is-proto/blob/main/utilities/packages/python/keycloak-utils/KeycloakRestUtilities/TokenManager.py

## Run all imports

Keep all your imports at the top of a notebook.  It allows for easier management.

In [20]:
import requests
import os
import sys
import json
   
sys.path.append("python_packages/rrap_packages")
    
from python_packages.rrap_packages.token_manager import DeviceFlowManager

## Define global variables

Similar to import we like to define notebook variable at the top and reuse them throughout the notebook

In [21]:
data_api = "https://data-api.testing.rrap-is.com"
registry_api = "https://registry-api.testing.rrap-is.com"
prov_api = "https://prov-api.testing.rrap-is.com"
auth_server = "https://auth.dev.rrap-is.com/auth/realms/rrap"

## Authentication steps

### Setup tokens using device authorisation flow against keycloak server

This could result in a browser window being opened if you don't have valid tokens cached in local storage.

In [22]:
# this caches the tokens
local_token_storage = ".tokens.json"

token_manager = DeviceFlowManager(
    stage="TEST",
    keycloak_endpoint=auth_server,
    local_storage_location=local_token_storage
)

Attempting to generate authorisation tokens.

Looking for existing tokens in local storage.

Validating found tokens

Found tokens valid, using.



### Authenticated call

In [ ]:
# Pull out the BearerAuth function (don't call it yet)
auth = token_manager.get_auth

# Setup postfix and endpoint
postfix = "/registry/items/list-all-datasets"
endpoint = data_api + postfix

# When making the request, use auth=auth() - this will ensure tokens are valid
# right at the point of using them, including potentially auto refreshing!
response = requests.get(endpoint, auth=auth())

print(json.dumps(response.json(), indent=4, sort_keys=True))

## Test your auth agains a endpoint 
https://registry-api.testing.rrap-is.com/docs#/Access%20check/check_general_access_check_access_check_general_access_get

In [25]:
#test general access endpoint
auth = token_manager.get_auth
base_url = "https://registry-api.testing.rrap-is.com"
url_query_string = "/check-access/check-general-access"
endpoint = base_url + url_query_string

response = requests.get(endpoint, auth=auth())

print(json.dumps(response.json(), indent=4, sort_keys=True))

{
    "access_token": "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICIzMVBQVkZZdE53a2JEcVZnNjFqX08xNXdiN2FRZU1BaExCbWdpY2VvY2dBIn0.eyJleHAiOjE2NTg0NTM5ODUsImlhdCI6MTY1ODQ1MzY4NSwiYXV0aF90aW1lIjoxNjU4NDUxODAzLCJqdGkiOiI1N2EyYTJjOC00ZTYxLTQ1YmItYmEwYS03MDRlMmU3MzVmM2UiLCJpc3MiOiJodHRwczovL2F1dGguZGV2LnJyYXAtaXMuY29tL2F1dGgvcmVhbG1zL3JyYXAiLCJhdWQiOlsicmVhbG0tbWFuYWdlbWVudCIsImFjY291bnQiXSwic3ViIjoiMzllOGQ2YjMtM2U5NS00MjU2LWEzMzEtNmZmZjkzYzU4NjFkIiwidHlwIjoiQmVhcmVyIiwiYXpwIjoiYWRtaW4tdG9vbHMiLCJzZXNzaW9uX3N0YXRlIjoiOGFkNzlhMjYtY2E1ZS00YTQxLTg4YjAtYzhkNGI4OThiYzkwIiwiYWNyIjoiMCIsInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJlbnRpdHktcmVnaXN0cnktYWRtaW4iLCJkYXRhLXN0b3JlLXdyaXRlIiwic3lzLWFkbWluLXJlYWQiLCJwcm92LWFkbWluIiwiaGFuZGxlLXdyaXRlIiwic3lzLWFkbWluLXdyaXRlIiwiaGFuZGxlLWFkbWluIiwicHJvdi1yZWFkIiwiZW50aXR5LXJlZ2lzdHJ5LXdyaXRlIiwiZGVmYXVsdC1yb2xlcy1ycmFwIiwicHJvdi13cml0ZSIsImVudGl0eS1yZWdpc3RyeS1yZWFkIiwib2ZmbGluZV9hY2Nlc3MiLCJkYXRhLXN0b3JlLWFkbWluIiwiZGF0YS1zdG9yZS1yZWFkIiwiaGFuZGxlLXJlYWQiLCJ1b